In [1]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib.pyplot as plt
import re
import seaborn as sns
sns.set(color_codes=True)

import tabula


In [2]:
df_pdf = tabula.read_pdf('kcp-mag-2021 v2.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    #area=[0, 0, 50, 100],
    relative_area=True,
    #multiple_tables=False
                        )

df = df_pdf[0]
for df_temp in df_pdf[1:]:
    df = df.append(df_temp,ignore_index=True)

In [3]:
df_mos = df[ df[1].str.contains('по программе|по совокупности', na=False) ].reset_index()
df_mos.tail()

print('дубликаты: {0}' .format( df_mos[[0,1]].duplicated().sum() ))

df_mos = df_mos[[0,1]].copy().drop_duplicates()
#df_mos

дубликаты: 0


In [4]:
df_one = df_mos[df_mos[1].str.contains('по программе магистратуры')]

def program_name(patern, tmp):
    tmp = tmp.replace('\r'," ").split(':',1)[1]
    #tmp = tmp.replace('  '," ")
    tmp = tmp.replace(re.findall(patern, tmp )[-1],'')
    " ".join(tmp.split())
    tmp = tmp.strip()
    return tmp

def facult_name(patern, tmp):
    tmp = tmp.replace('\r'," ")
    tmp = re.findall(patern, tmp )[-1].strip()
    return tmp

df_one_mos = pd.DataFrame([])
df_one_mos['код'] = df_one.iloc[:, 0]
df_one_mos['стандарт'] = df_one.iloc[:,1].map(lambda x: x.split('\r',1)[0])
df_one_mos['программа'] =  df_one.iloc[:,1].map(lambda x: program_name("\([ а-яa-zА-ЯA-Z0-9-: ]*\)", x))
df_one_mos['подразделение'] =  df_one.iloc[:,1].map(lambda x: facult_name("\(([ а-яa-zА-ЯA-Z0-9-: ]*)\)", x))
df_one_mos['программа'] = df_one_mos['программа'].map(lambda x: re.sub('[ё]', 'е', x))

#df_one_mos

In [5]:
df_multi = df_mos[df_mos[1].str.contains('по совокупности')]

def multi_transform(tmp):
    tmp_3 = tmp.iloc[1].split(':\r',1)[1]
    tmp_3 = " ".join(tmp_3.split())
    tmp_3 = re.split('\d.',tmp_3)[1:]
    tmp_3 = np.char.replace(tmp_3,'\r',' ')
    tmp_3 = np.char.replace(tmp_3,'.','')
    dic = {1: tmp.iloc[0],
           2: tmp.iloc[1].split('\r',1)[0],
           3: tmp_3
          }
    df_tmp = pd.DataFrame(dic)
    df_tmp[4] = df_tmp[3].map(lambda x: facult_name("\(([ а-яa-zА-ЯA-Z0-9-: ]*)\)", x))
    df_tmp[3] = df_tmp[3].map(lambda x: x.replace(re.findall("\([ а-яa-zА-ЯA-Z0-9-: ]*\)", x )[-1],'').strip() )
    return df_tmp

tmp = [ multi_transform(df_multi.loc[idx]) for idx in df_multi.index.values]

df_multi_mos = pd.concat(tmp, ignore_index=True)
df_multi_mos.columns = df_one_mos.columns.values
#display(df_multi_mos.head(3))
#display(df_multi_mos.tail(3))


In [6]:
df_moscow = pd.concat([df_one_mos, df_multi_mos], ignore_index=True)
df_moscow.drop_duplicates(inplace=True)
df_moscow.head(3)

код                стандарт  \
0  09.04.03  Прикладная информатика   
1  09.04.03  Прикладная информатика   
2  37.04.01              Психология   

                                           программа подразделение  
0                    Цифровые технологии в экономике    ФИТАД ЭМИТ  
1  Анализ данных и искусственный интеллект/ Data ...    ФИТАД ЭМИТ  
2                              Психология управления           ИОН

In [7]:
df_pdf = tabula.read_pdf('kcp-mag-filialu-2021 v2.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    #area=[0, 0, 50, 100],
    relative_area=True,
    #multiple_tables=False
                        )

df = df_pdf[0]
for df_temp in df_pdf[1:]:
    df = df.append(df_temp,ignore_index=True)
df.head(10)

searchfor = ['Направление','Общие',"очное"]
mask = df[1].str.contains('|'.join(searchfor), na=False)|df[2].str.contains('|'.join(searchfor), na=False)
df = df[~mask][[0,1]].copy()


mask_uslovie = df[0].str.contains("УСЛОВИЯ", na=False)


#df[0] = df[0].str.replace('УСЛОВИЯ ПОСТУПЛЕНИЯ:\r','')
df[0] = df[0].str.replace('УСЛОВИЯ ПОСТУПЛЕНИЯ:\r','')
df[0] = df[0].map(lambda x: x.split('– программы')[0]) # внимание тут разное тире в разных строках!!! 
df[0] = df[0].map(lambda x: x.split('- программы')[0]) # внимание тут разное тире в разных строках!!! 
df[0] = df[0].map(lambda x: x.split('- филиал')[0])
mask_list = np.array(df[mask_uslovie].index)


# with pd.option_context('display.max_rows', None):
#          display(df)

In [8]:
def func_dic(df_test):
    dic = {} 
    code = df_test[0]
    dic.update({0: code})
    tmp = df_test[1]
    napravlenie = tmp.split('\r',1)[0]
    dic.update({1: napravlenie})
    tmp = tmp.split('\r',2)[2]
    tmp = " ".join(tmp.split())
    tmp = re.split('\d.',tmp)[1:]
    tmp = np.char.replace(tmp,'\r',' ')
    tmp = np.char.replace(tmp,'.','')
    tmp = np.char.strip(tmp)
    dic.update({2: tmp})
    df_tmp = pd.DataFrame(dic)
    return df_tmp


df_multi_institute = []
mask_shape = mask_list.shape[0]
for i in range(mask_shape):
    if i < mask_shape-1:
        end = mask_list[i+1]-1
    else:
        end = df.index.max()
    
    df_fil = df.loc[mask_list[i]:end]
    
    tmp = [ func_dic(df_fil.loc[idx]) for idx in df_fil.iloc[1:].index.values]    
    df_multi = pd.concat(tmp, ignore_index=True)    
    df_multi[3] = df_fil.iloc[0][0].strip()

    
    df_multi = df_multi.drop_duplicates()
    df_multi_institute.append(df_multi)

df_multi_institute = pd.concat(df_multi_institute, ignore_index=True) 
df_multi_institute.columns = df_one_mos.columns.values

df_multi_institute['программа'] = df_multi_institute['программа'].str.replace('ё','е')

df_multi_institute.head(3)


код            стандарт                         программа  \
0  07.04.04  Градостроительство           Современная урбанистика   
1  37.04.01          Психология  Прикладная социальная психология   
2  38.04.01           Экономика         Экономика здравоохранения   

      подразделение  
0  Алтайский филиал  
1  Алтайский филиал  
2  Алтайский филиал

# Перечень всех программ магистратур Академии

In [9]:
magistratura = pd.concat([df_moscow,df_multi_institute],ignore_index=True)

with pd.option_context('display.max_rows', None):
         display(magistratura)

код                                    стандарт  \
0    09.04.03                      Прикладная информатика   
1    09.04.03                      Прикладная информатика   
2    37.04.01                                  Психология   
3    37.04.01                                  Психология   
4    38.04.01                                   Экономика   
5    38.04.01                                   Экономика   
6    38.04.01                                   Экономика   
7    38.04.01                                   Экономика   
8    38.04.01                                   Экономика   
9    38.04.01                                   Экономика   
10   38.04.01                                   Экономика   
11   38.04.01                                   Экономика   
12   38.04.01                                   Экономика   
13   38.04.01                                   Экономика   
14   38.04.01                                   Экономика   
15   38.04.01                                   Экономика   
16   38.04.01                                   Экономика   
17   38.04.01                                   Экономика   
18   38.04.01                                   Экономика   
19   38.04.01                                   Экономика   
20   38.04.01                                   Экономика   
21   38.04.02                                  Менеджмент   
22   38.04.02                                  Менеджмент   
23   38.04.02                                  Менеджмент   
24   38.04.02                                  Менеджмент   
25   38.04.02                                  Менеджмент   
26   38.04.02                                  Менеджмент   
27   38.04.02                                  Менеджмент   
28   38.04.02                                  Менеджмент   
29   38.04.02                                  Менеджмент   
30   38.04.02                                  Менеджмент   
31   38.04.02                                  Менеджмент   
32   38.04.02                                  Менеджмент   
33   38.04.02                                  Менеджмент   
34   38.04.02                                  Менеджмент   
35   38.04.02                                  Менеджмент   
36   38.04.02                                  Менеджмент   
37   38.04.02                                  Менеджмент   
38   38.04.02                                  Менеджмент   
39   38.04.02                                  Менеджмент   
40   38.04.02                                  Менеджмент   
41   38.04.02                                  Менеджмент   
42   38.04.02                                  Менеджмент   
43   38.04.02                                  Менеджмент   
44   38.04.02                                  Менеджмент   
45   38.04.02                                  Менеджмент   
46   38.04.02                                  Менеджмент   
47   38.04.02                                  Менеджмент   
48   38.04.02                                  Менеджмент   
49   38.04.02                                  Менеджмент   
50   38.04.02                                  Менеджмент   
51   38.04.02                                  Менеджмент   
52   38.04.02                                  Менеджмент   
53   38.04.02                                  Менеджмент   
54   38.04.02                                  Менеджмент   
55   38.04.02                                  Менеджмент   
56   38.04.02                                  Менеджмент   
57   38.04.02                                  Менеджмент   
58   38.04.02                                  Менеджмент   
59   38.04.02                                  Менеджмент   
60   38.04.02                                  Менеджмент   
61   38.04.03                       Управление персоналом   
62   38.04.03                       Управление персоналом   
63   38.04.04  Государственное и муниципальное управление   
64   38.04.04  Государственное и муниципально

In [10]:
tmp = magistratura[magistratura['код']=='38.04.01']

with pd.option_context('display.max_colwidth', None):
         display(tmp)

код   стандарт  \
4    38.04.01  Экономика   
5    38.04.01  Экономика   
6    38.04.01  Экономика   
7    38.04.01  Экономика   
8    38.04.01  Экономика   
9    38.04.01  Экономика   
10   38.04.01  Экономика   
11   38.04.01  Экономика   
12   38.04.01  Экономика   
13   38.04.01  Экономика   
14   38.04.01  Экономика   
15   38.04.01  Экономика   
16   38.04.01  Экономика   
17   38.04.01  Экономика   
18   38.04.01  Экономика   
19   38.04.01  Экономика   
20   38.04.01  Экономика   
158  38.04.01  Экономика   
169  38.04.01  Экономика   
170  38.04.01  Экономика   
176  38.04.01  Экономика   
199  38.04.01  Экономика   
200  38.04.01  Экономика   
205  38.04.01  Экономика   
211  38.04.01  Экономика   
220  38.04.01  Экономика   
221  38.04.01  Экономика   
222  38.04.01  Экономика   
232  38.04.01  Экономика   
233  38.04.01  Экономика   
258  38.04.01  Экономика   
267  38.04.01  Экономика   
274  38.04.01  Экономика   
275  38.04.01  Экономика   
276  38.04.01  Экономика   

                                                                         программа  \
4                                                              Экономика и финансы   
5                                                          Поведенческая экономика   
6                                               Системы больших данных в экономике   
7                                                               Цифровая экономика   
8                                                          Корпоративная экономика   
9                                                                Экономика и право   
10                                                        Экономика медиаиндустрии   
11                                   Экономика и контрольно-надзорная деятельность   
12                                                      Банки, финансы, инвестиции   
13         Бухгалтер-аналитик: корпоративная отчетность и финансовое моделирование   
14                    Финансы бизнеса: международные технологии учета и управления   
15                                        Финансы и стратегии устойчивого развития   
16                                                           Финансовая дипломатия   
17                               Управление инвестициями и инновациями в экономике   
18                                     Финансовый контроль и государственный аудит   
19   Финансовый контроль и государственный аудит (российско-французская программа)   
20                                         Государственное регулирование экономики   
158                                                      Экономика здравоохранения   
169                                             Экономика государственного сектора   
170                                       Управление бизнесом в цифровой экономике   
176                                                            Учет, анализ, аудит   
199                                             Экономика фирмы и отраслевые рынки   
200                                          Цифровая экономика и бизнес-аналитика   
205                                               Экономика и управление проектами   
211                                Управление стабильностью регионального развития   
220                        Анализ и моделирование экономичяеских процессов региона   
221                                      Экономический анализ и бухгалтерский учет   
222       Экономико-психологическое конструирование бизнес-процессов и организаций   
232                                                                Экономика фирмы   
233                                             Финансовые инструменты в экономике   
258                                                         Региональная экономика   
267                                            Экономика фирмы и отраслевых рынков   
274                             Финансовый контроль и аудит использования ресурсов   
275                                  Налогообложение и налоговое администриро

In [11]:
tmp = magistratura[magistratura['код']=='38.04.08']

with pd.option_context('display.max_colwidth', None):
         display(tmp)

код          стандарт  \
96   38.04.08  Финансы и кредит   
97   38.04.08  Финансы и кредит   
98   38.04.08  Финансы и кредит   
99   38.04.08  Финансы и кредит   
100  38.04.08  Финансы и кредит   
162  38.04.08  Финансы и кредит   
163  38.04.08  Финансы и кредит   
179  38.04.08  Финансы и кредит   
188  38.04.08  Финансы и кредит   
215  38.04.08  Финансы и кредит   
262  38.04.08  Финансы и кредит   

                                                         программа  \
96                                Прикладные корпоративные финансы   
97          Денежно-кредитное и финансовое регулирование экономики   
98                                              Финансовый инженер   
99                    Регулирование финансовых рынков и институтов   
100                                           Финансы и инвестиции   
162                        Государственные и муниципальные финансы   
163  Финансовая диагностика и организационные технологии в бизнесе   
179                                          Финансовый менеджмент   
188                                           Финансовая экономика   
215                                          Корпоративные финансы   
262                                          Финансовый менеджмент   

                           подразделение  
96                                  ВШФМ  
97                                   ФФБ  
98                                   ФФБ  
99                                  ИГСУ  
100                                 ИФУР  
162                     Алтайский филиал  
163                     Алтайский филиал  
179    Волгоградский институт управления  
188  Дальневосточный институт управления  
215    Нижегородский институт управления  
262    Среднерусский институт управления

# Перечень всех дубликатов программ магистратур Академии

In [12]:
tmp = magistratura[magistratura['программа'].duplicated(keep=False)].sort_values(by=['программа','код'])

with pd.option_context('display.max_colwidth', None):
         display(tmp)

код                                    стандарт  \
95   38.04.05                          Бизнес-информатика   
242  38.04.05                          Бизнес-информатика   
72   38.04.04  Государственное и муниципальное управление   
219  38.04.04  Государственное и муниципальное управление   
250  38.04.04  Государственное и муниципальное управление   
108  40.04.01                               Юриспруденция   
180  40.04.01                               Юриспруденция   
30   38.04.02                                  Менеджмент   
206  38.04.02                                  Менеджмент   
44   38.04.02                                  Менеджмент   
190  38.04.02                                  Менеджмент   
29   38.04.02                                  Менеджмент   
201  38.04.02                                  Менеджмент   
259  38.04.02                                  Менеджмент   
104  40.04.01                               Юриспруденция   
174  40.04.01                               Юриспруденция   
189  40.04.01                               Юриспруденция   
217  40.04.01                               Юриспруденция   
229  40.04.01                               Юриспруденция   
256  40.04.01                               Юриспруденция   
270  40.04.01                               Юриспруденция   
203  40.04.01                               Юриспруденция   
230  40.04.01                               Юриспруденция   
167  38.04.04  Государственное и муниципальное управление   
173  38.04.04  Государственное и муниципальное управление   
191  38.04.04  Государственное и муниципальное управление   
249  38.04.04  Государственное и муниципальное управление   
209  38.04.04  Государственное и муниципальное управление   
214  38.04.04  Государственное и муниципальное управление   
226  38.04.04  Государственное и муниципальное управление   
178  38.04.04  Государственное и муниципальное управление   
202  38.04.04  Государственное и муниципальное управление   
69   38.04.04  Государственное и муниципальное управление   
159  38.04.04  Государственное и муниципальное управление   
186  38.04.04  Государственное и муниципальное управление   
192  38.04.04  Государственное и муниципальное управление   
194  38.04.04  Государственное и муниципальное управление   
196  38.04.04  Государственное и муниципальное управление   
257  38.04.04  Государственное и муниципальное управление   
269  38.04.04  Государственное и муниципальное управление   
273  38.04.04  Государственное и муниципальное управление   
54   38.04.02                                  Менеджмент   
212  38.04.02                                  Менеджмент   
268  38.04.02                                  Менеджмент   
234  38.04.02                                  Менеджмент   
68   38.04.04  Государственное и муниципальное управление   
185  38.04.03                       Управление персоналом   
225  38.04.03                       Управление персоналом   
179  38.04.08                            Финансы и кредит   
262  38.04.08                            Финансы и кредит   
145  40.04.01                               Юриспруденция   
265  40.04.01                               Юриспруденция   

                                                             программа  \
95                                                    Бизнес-аналитика   
242                                                   Бизнес-аналитика   
72                          Государственная служба и кадровая политика   
219                         Государственная служба и кадровая политика   
250                         Государственная служба и кадровая политика   
108     Гражданское право, семейное право, международное частное право   
180     Гражданское право, семейное право, международное частное право   
30                                           Деловое администрирование   
206                                          Деловое администрирование   
44          Корпоративное управление

# Перечень всех программ магистратур Академии по ЭКОНОМИКЕ

In [13]:
tmp = magistratura[magistratura['код']=='38.04.01'].sort_values(by='программа')

with pd.option_context('display.max_colwidth', None):
         display(tmp)

код   стандарт  \
220  38.04.01  Экономика   
12   38.04.01  Экономика   
13   38.04.01  Экономика   
20   38.04.01  Экономика   
8    38.04.01  Экономика   
276  38.04.01  Экономика   
275  38.04.01  Экономика   
5    38.04.01  Экономика   
258  38.04.01  Экономика   
6    38.04.01  Экономика   
170  38.04.01  Экономика   
17   38.04.01  Экономика   
211  38.04.01  Экономика   
176  38.04.01  Экономика   
16   38.04.01  Экономика   
233  38.04.01  Экономика   
274  38.04.01  Экономика   
18   38.04.01  Экономика   
19   38.04.01  Экономика   
14   38.04.01  Экономика   
15   38.04.01  Экономика   
7    38.04.01  Экономика   
200  38.04.01  Экономика   
169  38.04.01  Экономика   
158  38.04.01  Экономика   
11   38.04.01  Экономика   
9    38.04.01  Экономика   
205  38.04.01  Экономика   
4    38.04.01  Экономика   
10   38.04.01  Экономика   
232  38.04.01  Экономика   
199  38.04.01  Экономика   
267  38.04.01  Экономика   
222  38.04.01  Экономика   
221  38.04.01  Экономика   

                                                                         программа  \
220                        Анализ и моделирование экономичяеских процессов региона   
12                                                      Банки, финансы, инвестиции   
13         Бухгалтер-аналитик: корпоративная отчетность и финансовое моделирование   
20                                         Государственное регулирование экономики   
8                                                          Корпоративная экономика   
276                                                              Мировая экономика   
275                                  Налогообложение и налоговое администрирование   
5                                                          Поведенческая экономика   
258                                                         Региональная экономика   
6                                               Системы больших данных в экономике   
170                                       Управление бизнесом в цифровой экономике   
17                               Управление инвестициями и инновациями в экономике   
211                                Управление стабильностью регионального развития   
176                                                            Учет, анализ, аудит   
16                                                           Финансовая дипломатия   
233                                             Финансовые инструменты в экономике   
274                             Финансовый контроль и аудит использования ресурсов   
18                                     Финансовый контроль и государственный аудит   
19   Финансовый контроль и государственный аудит (российско-французская программа)   
14                    Финансы бизнеса: международные технологии учета и управления   
15                                        Финансы и стратегии устойчивого развития   
7                                                               Цифровая экономика   
200                                          Цифровая экономика и бизнес-аналитика   
169                                             Экономика государственного сектора   
158                                                      Экономика здравоохранения   
11                                   Экономика и контрольно-надзорная деятельность   
9                                                                Экономика и право   
205                                               Экономика и управление проектами   
4                                                              Экономика и финансы   
10                                                        Экономика медиаиндустрии   
232                                                                Экономика фирмы   
199                                             Экономика фирмы и отраслевые рынки   
267                                            Экономика фирмы и отраслевых рынков   
222       Экономико-психологическое конструирование бизнес-процессов и органи

# Перечень всех программ магистратур Академии по ФИНАНСАМ и КРЕДИТУ

In [14]:
tmp = magistratura[magistratura['код']=='38.04.08'].sort_values(by='программа')

with pd.option_context('display.max_colwidth', None):
         display(tmp)

код          стандарт  \
162  38.04.08  Финансы и кредит   
97   38.04.08  Финансы и кредит   
215  38.04.08  Финансы и кредит   
96   38.04.08  Финансы и кредит   
99   38.04.08  Финансы и кредит   
163  38.04.08  Финансы и кредит   
188  38.04.08  Финансы и кредит   
98   38.04.08  Финансы и кредит   
179  38.04.08  Финансы и кредит   
262  38.04.08  Финансы и кредит   
100  38.04.08  Финансы и кредит   

                                                         программа  \
162                        Государственные и муниципальные финансы   
97          Денежно-кредитное и финансовое регулирование экономики   
215                                          Корпоративные финансы   
96                                Прикладные корпоративные финансы   
99                    Регулирование финансовых рынков и институтов   
163  Финансовая диагностика и организационные технологии в бизнесе   
188                                           Финансовая экономика   
98                                              Финансовый инженер   
179                                          Финансовый менеджмент   
262                                          Финансовый менеджмент   
100                                           Финансы и инвестиции   

                           подразделение  
162                     Алтайский филиал  
97                                   ФФБ  
215    Нижегородский институт управления  
96                                  ВШФМ  
99                                  ИГСУ  
163                     Алтайский филиал  
188  Дальневосточный институт управления  
98                                   ФФБ  
179    Волгоградский институт управления  
262    Среднерусский институт управления  
100                                 ИФУР

In [15]:
# magistratura.to_excel("mag_2021.xlsx")